In [1]:
import os

file_names = []
corpus = []
for file_name in os.listdir('./data/corpus'):
    with open(f'./data/corpus/{file_name}', 'r', encoding="utf8") as f:
        doc = f.readlines()

    file_names.append(" ".join(file_name.split("-")))
    corpus.append(" ".join(doc))

In [2]:
corpus[0]

'https://tamanhhospital.vn/alzheimer/\n \n                     Trang chủ > \n CHUYÊN MỤC BỆNH HỌC > \n Thần kinh > \n Bệnh Alzheimer: Nguyên nhân, triệu chứng, điều trị và phòng ngừa\n <h1>Bệnh Alzheimer: Nguyên nhân, triệu chứng, điều trị và phòng ngừa</h1>\n <i><strong>Bệnh Alzheimer</strong> là nguyên nhân hàng đầu gây nên tình trạng sa sút trí tuệ, ảnh hưởng đến suy nghĩ và hành vi của con người. Không có cách nào để đảo ngược quá trình tiến triển bệnh, nhưng việc phát hiện và điều trị sớm có thể giúp nâng cao chất lượng cuộc sống cho người bệnh.\n </i>  Thống kê cho thấy trên thế giới có ít nhất 50 triệu người đang sống chung với bệnh Alzheimer hoặc các hội chứng sa sút trí tuệ khác. Theo Liên Hợp Quốc, con số đó nhiều hơn dân số của Columbia, nếu không có những đột phá trong việc chẩn đoán và hạn chế bệnh, tỷ lệ này có thể vượt quá 152 triệu người vào năm 2050.\n <h3>Mục lục</h3>\n Bệnh Alzheimer là gì?\n Lịch sử phát hiện bệnh\n Nguyên nhân\n Ai là đối tượng dễ mắc bệnh?\n Triệu

In [3]:
file_names[0]

'alzheimer'

In [4]:
import re
import string

def remove_url(text):
    return re.sub(r"http\S+", "", text)

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

codes = ["&#8230, &#8211", "&#038"]
def remove_special_token(text):
    for code in codes:
        text = text.replace(code, " ")
    return text

def remove_punctation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

def preprocess_text(text, remove_punc=True):
    if remove_punc:
        return " ".join(remove_punctation(remove_special_token(remove_html_tags(remove_url(text)))).split()).lower()
    else:
        return " ".join(remove_special_token(remove_html_tags(remove_url(text))).split()).lower()

In [5]:
codes = ["&#8211", "&#038"]
titles = []
for file_name, doc in zip(file_names, corpus):
    raw = doc.split("\n")[5]
    title = doc.split("\n")[5].split(":")[0].split("?")[0]
    for code in codes:
        title = title.replace(code, " ")
    title = preprocess_text(title)
    titles.append(title)

In [6]:
titles[0]

'bệnh alzheimer'

In [7]:
words = [
    [word for word in preprocess_text(doc).split()]
    for doc in titles
]

In [8]:
words[0]

['bệnh', 'alzheimer']

In [9]:
def split_doc(doc):
    paragraphs = doc.split("<h3>Mục lục</h3>")[1].split("<h2>")
    menu = paragraphs[0]
    paragraphs = paragraphs[1:]
    process_paragraphs = []
    for paragraph in paragraphs:
        if "hệ thống bệnh viện đa khoa tâm anh" in paragraph.lower():
            process_paragraph = paragraph.lower().split("hệ thống bệnh viện đa khoa tâm anh")[0]
            process_paragraph = paragraph[ : len(process_paragraph)]
        else:
            process_paragraph = paragraph
        process_paragraphs.append(process_paragraph)
    return menu, process_paragraphs

In [10]:
all_passage = []

for idx, doc in enumerate(corpus):
  menu, process_paragraphs = split_doc(doc)
  dict_paragraphs = {}
  ten_benh = titles[idx]
  tenbenh = ""
  if "là" in ten_benh:
    # Tách chuỗi thành danh sách các từ
    words = ten_benh.split()
    
    # Tìm chỉ số của từ "là" trong danh sách các từ
    index_of_la = words.index("là") if "là" in words else -1
    
    # Nếu "là" được tìm thấy, chỉ lấy các từ trước "là"
    if index_of_la != -1:
      tenbenh = ' '.join(words[:index_of_la])
  else:
    tenbenh = ten_benh

  dict_paragraphs.update({"tên_bệnh": tenbenh})
  
  for item in process_paragraphs:
    a, b = item.split("</h2>")
    
    key_list = ["nguyên nhân", "triệu chứng", "điều trị", "phòng ngừa"]
    
    if any(key in a.lower() for key in key_list):  
      print(preprocess_text(a))
      key = next(key for key in key_list if key in a.lower())  # Find the matching key
      ks = key.split(' ')
      dict_paragraphs.update({"_".join(ks): preprocess_text(b)})
  
  all_passage.append({"index": {"_id": idx, "_index": "medical"}})
  all_passage.append(dict_paragraphs)

nguyên nhân
triệu chứng thường gặp của hội chứng alzheimer
biện pháp điều trị
nguyên nhân ăn không tiêu
cách phòng ngừa ăn không tiêu
nguyên nhân áp xe hậu môn
triệu chứng áp xe hậu môn
điều trị áp xe hậu môn
cách phòng ngừa áp xe hậu môn
nguyên nhân gây ép xe phổi
triệu chứng thường gặp
phương pháp điều trị bệnh áp xe phổi
phòng ngừa áp xe phổi bằng cách nào
nguyên nhân gây loét bàn chân tiểu đường
triệu chứng loét bàn chân bệnh tiểu đường
điều trị bệnh lý bàn chân tiểu đường
phòng ngừa biến chứng bàn chân đái tháo đường
nguyên nhân gây băng huyết ‌ở phụ nữ sau sinh
cách điều trị băng huyết sau sinh
cách phòng ngừa băng huyết sau sinh
triệu chứng bàng quang tăng hoạt
nguyên nhân khiến bàng quang hoạt động quá mức
phương pháp điều trị bàng quang tăng hoạt
phòng ngừa bàng quang tăng hoạt
nguyên nhân gây bế sản dịch sau khi sinh
cách phòng ngừa bế sản dịch như thế nào
giải pháp phòng ngừa và điều trị tắc sản dịch sau sinh mổ thường tại bvđk tâm anh
dấu hiệu và triệu chứng nhận biết
nguyê

triệu chứng thường gặp
nguyên nhân khiến cổ tay bị đau sau khi sinh con
điều trị đau cổ tay sau sinh
biện pháp phòng ngừa
nguyên nhân gây đau dạ dày
phòng ngừa đau dạ dày đau bao tử
nguyên nhân gây đau đầu
triệu chứng của cơn đau đầu
các phương pháp điều trị chứng đau đầu
phòng ngừa cơn đau nhức đầu
triệu chứng bất thường ở đầu gối khi ngồi hoặc đứng
nguyên nhân đau đầu gối khi đứng lên ngồi xuống
điều trị đau đầu gối khi đứng lên ngồi xuống
cách phòng ngừa đau đầu gối khi đứng
triệu chứng đau nửa đầu migraine
nguyên nhân gây đau đầu migraine
cách điều trị đau nửa đầu migraine
cách phòng ngừa bệnh đau đầu migraine
nguyên nhân đau nửa đầu sau gáy
triệu chứng đau đầu sau gáy nguy hiểm
cách điều trị đau nửa đầu sau gáy như thế nào
triệu chứng đau đầu vận mạch
nguyên nhân đau đầu vận mạch
cách điều trị đau đầu vận mạch
triệu chứng đau đỉnh đầu
nguyên nhân đau đỉnh đầu thường gặp
các nguyên nhân gây đau đỉnh đầu khác
cách điều trị nhức đỉnh đầu
nguyên nhân chuyển phôi thất bại
17 nguyên nhâ

In [11]:
import json
# Lưu dữ liệu vào file JSON
with open("output_final.json", "w", encoding="utf-8") as json_file:
    for passage in all_passage:
        json.dump(passage, json_file, ensure_ascii=False)
        json_file.write("\n")